""" Pending tasks:  
1) Add texture to the mesh
2) Test reconstruction qualities for various hyperparamters of each reconstruction algorithm  

Optional tasks:
1) Include alpha_shapes reconstruction technique
2) Understand difference between TetraMesh and Triangle mesh quality and processing times

Sources: 
1) Surface reconstruction tutorial: https://towardsdatascience.com/5-step-guide-to-generate-3d-meshes-from-point-clouds-with-python-36bad397d8ba
2) Open3D tutorials: https://www.youtube.com/playlist?list=PLkmvobsnE0GEZugH1Di2Cr_f32qYkv7aN 
3) Open3D: http://www.open3d.org/docs/release/index.html
4) laspy: https://laspy.readthedocs.io/en/latest/

## 1) Import Open3D and laspy

In [1]:
#%pip install laspy
#%pip install open3d

Note: you may need to restart the kernel to use updated packages.


The filename, directory name, or volume label syntax is incorrect.


In [1]:
import numpy as np
import open3d as o3d
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt

#import wget -> In case you want to download point cloud and meshes from url's

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Special case of using a las file (preserves true coordinates for x,y,z)

In [2]:
import laspy as lp

las_file = "point_cloud.las"
point_cloud = lp.read(las_file)

In [80]:
def header_info(lasfile=point_cloud):
    print(f"File version is: {lasfile.header._version}")
    print(f"Point format id: {lasfile.header.point_format.id}")
    print(f"Includes the following dimensions: {[i for i in lasfile.header._point_format.dimension_names]}")
    print(f"Includes the following extra dimensions: {[i for i in lasfile.header._point_format.extra_dimension_names]}")

def vlrs_info(lasfile=point_cloud):
    [print(_) for _ in lasfile.vlrs]


header_info(point_cloud)
vlrs_info(point_cloud)


File version is: 1.2
Point format id: 3
Includes the following dimensions: ['X', 'Y', 'Z', 'intensity', 'return_number', 'number_of_returns', 'scan_direction_flag', 'edge_of_flight_line', 'classification', 'synthetic', 'key_point', 'withheld', 'scan_angle_rank', 'user_data', 'point_source_id', 'gps_time', 'red', 'green', 'blue']
Includes the following extra dimensions: []
<GeoKeyDirectoryVlr(12 geo_keys)>
<GeoDoubleParamsVlr([c_double(6378137.0), c_double(298.257223563), c_double(0.0)])>
<GeoAsciiParamsVlr(['CGRS93 / Cyprus Local Transverse Mercator|GCS Name = CGRS93|Datum = unknown|Ellipsoid = unretrievable - using WGS84|Primem = Greenwich||', ''])>


In [3]:
points = np.vstack((point_cloud.x, point_cloud.y, point_cloud.z)).transpose()
colors = np.vstack((point_cloud.red, point_cloud.green, point_cloud.blue)).transpose()

las_pcd = o3d.geometry.PointCloud()
las_pcd.points = o3d.utility.Vector3dVector(points)
las_pcd.colors = o3d.utility.Vector3dVector(colors / 65535)

In [74]:
def write_pointcloud(pcd=las_pcd, file_name = "las_pcd.ply"):
    try:
        o3d.io.write_point_cloud(filename = file_name, pointcloud=las_pcd) 
    except Exception as e:
        print(e)

In [75]:
write_pointcloud()

## 2) Load and read the point cloud

### translating to correct coordiante system

In [103]:
import math
yaw = 30 # Example value
cropped_pcd = o3d.io.read_point_cloud("./cropped_pointcloud.ply", format="ply")

x = [_[0] for _ in np.asarray(cropped_pcd.points)]
y = [_[1] for _ in np.asarray(cropped_pcd.points)]
z = [_[2] for _ in np.asarray(cropped_pcd.points)]

# original_x = x / math.acos(yaw) + y / math.asin(yaw)
# original_y = -x / math.asin(yaw) + y / math.acos(yaw)

In [4]:
file_name = "las_pcd.ply"
original_pointcloud = o3d.io.read_point_cloud(file_name, format="ply")
pointcloud_array = np.asarray(original_pointcloud.points)

n=5
print(f"Point cloud dimensions {pointcloud_array.shape}")
print(f"First {n} datapoints:")
print(pointcloud_array[:n])


Point cloud dimensions (7828904, 3)
First 5 datapoints:
[[2.37511591e+05 3.90693672e+05 1.59812600e+02]
 [2.37514860e+05 3.90691483e+05 1.60057900e+02]
 [2.37514647e+05 3.90691871e+05 1.60180100e+02]
 [2.37512661e+05 3.90694604e+05 1.60139600e+02]
 [2.37515006e+05 3.90691411e+05 1.60182800e+02]]


## 3) Voxel Downsampling our point cloud

In [5]:
downsampled_pcd = original_pointcloud.voxel_down_sample(voxel_size=1.0)
downsampled_array = np.asarray(downsampled_pcd.points)
print(downsampled_array.shape)
print(f"Size reduced to {round(100 * downsampled_array.shape[0]/pointcloud_array.shape[0],2)}% of original pointcloud")
o3d.visualization.draw_geometries([downsampled_pcd])

(46312, 3)
Size reduced to 0.59% of original pointcloud
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


In [83]:
#write_pointcloud(downsampled_array, "compressed_las.ply")

### Poisson disk sampling and uniform sampling (wont be used for now)

In [ ]:
# pcd = mesh.sample_points_poisson_disk(number_of_points=100_000, init_factor=5)
# o3d.visualization.draw_geometries([pcd])

# pcd = mesh.sample_points_uniformly(number_of_points=2500)
# o3d.visualization.draw_geometries([pcd]) 

## 4) Crop the point cloud

In [4]:
def demo_crop_geometry(pcd):
    print("Demo for manual geometry cropping")
    print(
        "1) Press 'Y' twice to align geometry with negative direction of y-axis"
    )
    print("2) Press 'K' to lock screen and to switch to selection mode")
    print("3) Drag for rectangle selection,")
    print("   or use ctrl + left click for polygon selection")
    print("4) Press 'C' to get a selected geometry and to save it")
    print("5) Press 'F' to switch to freeview mode")
    o3d.visualization.draw_geometries_with_editing([pcd])

Demo for manual geometry cropping
1) Press 'Y' twice to align geometry with negative direction of y-axis
2) Press 'K' to lock screen and to switch to selection mode
3) Drag for rectangle selection,
   or use ctrl + left click for polygon selection
4) Press 'C' to get a selected geometry and to save it
5) Press 'F' to switch to freeview mode


In [ ]:
#demo_crop_geometry(downsampled_pcd)

## 5) Visualize cropped pointcloud

In [3]:
cropped_pointcloud = o3d.io.read_point_cloud("./cropped_ucy.ply", format="ply")
o3d.visualization.draw_geometries([cropped_pointcloud])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


## 6) Compute Normals

### Compute normals and estimate average distance between points

In [4]:
pcd = o3d.geometry.PointCloud()
pcd.points = cropped_pointcloud.points
pcd.colors = cropped_pointcloud.colors
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 3 * avg_dist

## 7) Mesh reconstruction

### Ball pivoting algorithm

In [5]:
bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd, o3d.utility.DoubleVector([radius, radius * 2]))

### Poisson surface reconstruction

In [9]:
poisson_mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=8, width=0, scale=1.1, linear_fit=False)
vertices_to_remove = densities < np.quantile(densities, 0.1)
poisson_mesh.remove_vertices_by_mask(vertices_to_remove)
o3d.visualization.draw_geometries([poisson_mesh], point_show_normal = True)

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle

### Further Postprocessing to remove unecessary triangles and vertices

In [11]:
def postprocess(mesh, pcd=pcd):
    """simplify surface, remove degenerate triangles, vertices and non-manifold edges"""
    mesh.compute_vertex_normals()
    mesh = mesh.simplify_quadric_decimation(100000)
    mesh.remove_degenerate_triangles()
    mesh.remove_duplicated_triangles()
    mesh.remove_duplicated_vertices()
    mesh.remove_non_manifold_edges()
    bbox = pcd.get_axis_aligned_bounding_box()
    mesh = mesh.crop(bbox)
    print(f"Resulting mesh has: {mesh.triangles} triangles")
    return mesh

def density_estimation(mesh):
    """Removes low definition areas of the mesh, NO NEED TO USE FOR NOW AS FILTERING IS DONE ABOVE"""
    densities = np.asarray(mesh)
    density_colours = plt.get_cmap("plasma")(densities - densities.min()) /(densities.max()-densities.min())
    density_colours = density_colours[:,:3]
    density_mesh = o3d.geometry.TriangleMesh()
    density_mesh.vertices = mesh.vertices
    density_mesh.triangles = mesh.triangles
    density_mesh.triangle_normals = mesh.triangle_normals
    density_mesh.vertex_colors = o3d.utility.Vector3dVector(density_colours)
    return densities


### Check mesh vertices, normals

In [ ]:
def check_mesh(mesh):
    if not mesh.has_vertex_normals(): 
        print("No vertex normals")   
        mesh.compute_vertex_normals()
    if not mesh.has_triangle_normals(): 
        print("No triangle normals")
        mesh.compute_triangle_normals()
        sampled_points = mesh.sample_points_poisson_disk(82747)
    print(len(mesh.vertices),len(mesh.triangles))

# check_mesh(bpa_mesh)
# check_mesh(poisson_mesh)

In [27]:
def surface_reconstruction(pcd, algorithm, alpha=0.03):
    """ALPHA SHAPES ONLY WORKS WITH TETRAMESH, WILL INSPECT SOON"""
    methods ={
        "alpha_shapes": o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha),
        "ball_pivoting": o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd),
        "poisson": o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd)
    }
    mesh = methods[algorithm]
    mesh.compute_vertex_normals()
    o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)
    return mesh

### Visualise the final mesh reconstruction

In [12]:
# Visualise if postprocessing is improving the mesh 
poisson_mesh = postprocess(poisson_mesh)
o3d.visualization.draw_geometries([poisson_mesh], point_show_normal = True)

Resulting mesh has: std::vector<Eigen::Vector3i> with 89500 elements.
Use numpy.asarray() to access data. triangles
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


In [7]:
bpa_mesh = postprocess(bpa_mesh)
o3d.visualization.draw_geometries([bpa_mesh], point_show_normal = True)

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


### Download or read a premade mesh, used when working with .obj files

In [4]:
def download_ready_mesh(url = "./insert_url_to_your_mesh/mesh_name.obj"):
    import wget
    # In case you want to download your mesh from a url
    return wget.download(url)

def read_obj_mesh(mesh ="./ucy.obj"):
    pcd_mesh = o3d.io.read_triangle_mesh(mesh)
    if pcd_mesh.is_empty(): exit()
    else:
        return pcd_mesh

## 8) Write files to a Excel or generate a new ".PLY" file

### Write points to an Excel File

In [38]:
def excel_logger(file_name="./cropped_point_cloud.ply", excel_file = "light_cloud.pcd.xlsx"):
    """EDIT TO INCLUDE OTHER DATA ENCODED IN THE PLY SUCH AS COLORS"""
    pcd = o3d.io.read_point_cloud(file_name, format="ply")
    pcd_array = np.asarray(original_pointcloud.points)
    data_transformer = pd.DataFrame(data = pcd_array, columns  = ["X", "Y", "Z", "Normals_X", "Normals_Y", "Normals_Z"], dtype=np.float32)
    data_transformer.to_excel(excel_file)

## 9) Define a plot function for point cloud and mesh

In [16]:
def draw(geometries):
    graph_obj = []

    for gm in geometries:
        geometry_type = gm.get_geometry_type()
        
        if geometry_type == o3d.geometry.Geometry.Type.PointCloud:
            pts = np.asarray(gm.points)
            clr = None  #for colors
            if gm.has_colors():
                clr = np.asarray(gm.colors)
            elif gm.has_normals():
                clr = (0.5, 0.5, 0.5) + np.asarray(gm.normals) * 0.5
            else:
                gm.paint_uniform_color((1.0, 0.0, 0.0))
                clr = np.asarray(gm.colors)

            sc = go.Scatter3d(x=pts[:,0], y=pts[:,1], z=pts[:,2], mode='markers', marker=dict(size=1, color=clr))
            graph_obj.append(sc)

        if geometry_type == o3d.geometry.Geometry.Type.TriangleMesh:
            tri = np.asarray(gm.triangles)
            vert = np.asarray(gm.vertices)
            clr = None
            if gm.has_triangle_normals():
                clr = (0.5, 0.5, 0.5) + np.asarray(gm.triangle_normals) * 0.5
                clr = tuple(map(tuple, clr))
            else:
                clr = (1.0, 0.0, 0.0)
            
            mesh = go.Mesh3d(x=vert[:,0], y=vert[:,1], z=vert[:,2], i=tri[:,0], j=tri[:,1], k=tri[:,2], facecolor=clr, opacity=0.50)
            graph_obj.append(mesh)
        
    fig = go.Figure(
        data=graph_obj,
        layout=dict(
            scene=dict(
                xaxis=dict(visible=False),
                yaxis=dict(visible=False),
                zaxis=dict(visible=False)
            )
        )
    )
    fig.show()

In [26]:
# o3d.visualization.draw_geometries = draw # replace function
# o3d.visualization.draw_geometries([pcd])
# o3d.visualization.draw_geometries([pcd_mesh])